In [1]:
import pickle

# Loading the data.

In [64]:
features = []
targets  = []

In [72]:
with open("data/pointing_at_camera_data", "rb") as file:
    data = pickle.load(file)

features += data[0]
targets  += [3 for i in data[1]]

## Labels: 
- 0: open hand
- 1: closed hand
- 2: swipe 
- 3: pointing at the camera

# Shuffling the data

In [109]:
import numpy as np

In [126]:
temp = list(zip(features, targets))
np.random.shuffle(temp)

In [127]:
shuffled_features, shuffled_targets = zip(*temp)

# Training a simple test XGBoost

In [129]:
from xgboost import XGBClassifier

In [222]:
xgb_model = XGBClassifier(objective = "multi:softprob")

## Formatting the data to training

In [223]:
features_array = np.array(shuffled_features)
features_reshaped = features_array.reshape((features_array.shape[0], features_array.shape[1]*features_array.shape[2]))

targets_array = np.array(shuffled_targets)

In [224]:
xgb_model.fit(features_reshaped, targets_array)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

# Taking a look at the shape for predicitons

In [173]:
test_feat = features_reshaped[0]
test_target = targets_array[0]

In [190]:
pred = xgb_model.predict_proba(test_feat.reshape(1, -1))

In [183]:
prob = pred.max()
gestures = pred.argmax()

In [185]:
if prob >0.9:
    print(f"Do gesture {gestures}")

Do gesture 2


# Testing live cam

In [191]:
import cv2

from kotoamatsukami.src.domain.hand_detector import HandDetector

In [194]:
stream = cv2.VideoCapture(0)
detector = HandDetector(max_hands = 1, detection_con = 0.8)


i = 0

while True:

    _grabbed, frame = stream.read()

    if cv2.waitKey(1) == ord("x"):
        break

    hands, image = detector.find_hands(frame)

    if len(hands) != 0:
        for hand in hands:
            feat = []

            for landmark in hand["landmarks"]:
                feat += landmark
            
            prediction = xgb_model.predict_proba([feat])
            i += 1

            if prediction.max() > 0.999:
                print(f"Catch the {prediction.argmax()} gesture - {i}")

    cv2.imshow("Video", image)

stream.release()
cv2.destroyAllWindows()

Catch the 0 gesture - 59
Catch the 0 gesture - 60
Catch the 0 gesture - 61
Catch the 0 gesture - 62
Catch the 0 gesture - 63
Catch the 0 gesture - 64
Catch the 0 gesture - 65
Catch the 0 gesture - 66
Catch the 0 gesture - 67
Catch the 0 gesture - 68
Catch the 0 gesture - 69
Catch the 0 gesture - 70
Catch the 0 gesture - 71
Catch the 0 gesture - 72
Catch the 0 gesture - 73
Catch the 0 gesture - 74
Catch the 0 gesture - 75
Catch the 0 gesture - 76
Catch the 0 gesture - 77
Catch the 0 gesture - 78
Catch the 0 gesture - 79
Catch the 0 gesture - 80
Catch the 0 gesture - 81
Catch the 0 gesture - 82
Catch the 0 gesture - 83
Catch the 0 gesture - 84
Catch the 0 gesture - 85
Catch the 0 gesture - 86
Catch the 0 gesture - 87
Catch the 0 gesture - 88
Catch the 0 gesture - 89
Catch the 0 gesture - 90
Catch the 0 gesture - 91
Catch the 0 gesture - 92
Catch the 0 gesture - 97
Catch the 0 gesture - 98
Catch the 0 gesture - 99
Catch the 0 gesture - 100
Catch the 0 gesture - 101
Catch the 0 gesture - 1

# Saving the model

In [225]:
with open("models/initial_4_classes_xgb_model", "wb") as file:
    pickle.dump(xgb_model, file)